In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 4.3 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=a3784ffde7cb95c2350ae6c818fb24c4a4d484df9586fd506ea0fea5cb597818
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv   a0606.csv	        heatmap531.ipynb
 a0530.csv   a0607.csv	        Heat_Map_Online_language.ipynb
 a0531.csv   a0608.csv	        __MACOSX
 a0601.csv   csv	        non_violant1.csv
 a0602.csv   csv.zip	        non_violant2.csv
 a0603.csv   dataframe_csv      violant.csv
 a0604.csv   Github	       '新建 Microsoft PowerPoint 演示文稿.pptx'
 a0605.csv   heatmap530.ipynb


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_609 = 'csv/US06_09.csv'

In [8]:
ct = pd.read_csv(ct_path_609)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,189346456615_10157352875286616,Facebook,2020-06-09 21:05:38,2021-07-07 12:22:24,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",hot1079.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/452p8Y6,https://www.facebook.com/189346456615/posts/10...,15541,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,1,2,1,1,0,0,428743,HOT 107.9 Syracuse,hot1079syracuse,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,HOT 107.9 | All The Hits For Syracuse\nElvis D...,True,2009-12-04 17:48:04,en,0,428743|10157352875286616
1,48406298570_10158377191118571,Facebook,2020-06-09 21:05:38,2021-09-04 02:11:57,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",kiss1023.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/qcie1mk,https://www.facebook.com/48406298570/posts/101...,77246,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,1,1,1,1,0,1,0,0,0,156408,KISS 102-3,Kiss1023,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,The Capital Region's Home for Elvis Duran in t...,True,2009-01-28 22:01:34,en,0,156408|10158377191118571
2,838752389553295_2902666793161834,Facebook,2020-06-09 21:05:37,2021-05-13 06:06:25,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",thebeatgulfcoast.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/iCfLMAy,https://www.facebook.com/838752389553295/posts...,1726,-0.928571,1.0,0,0,0,1,0,0,0,0,0,0,1,2,1,2,1,2,2,3,0,1,3261001,The Beat Gulf Coast,TheBeat1073,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Mobile's Home for Hip Hop n' R & B,False,2015-08-31 17:09:00,en,0,3261001|2902666793161834
3,11236264990_10151291314189991,Facebook,2020-06-09 21:05:37,2021-07-08 05:06:04,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",wfmf.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/QjnkvRM,https://www.facebook.com/11236264990/posts/101...,9200,-20.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,1,2,1,0,1,39088,102.5 WFMF,1025wfmf,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Baton Rouge's #1 Hit Music station with Kidd K...,True,2008-08-02 02:16:16,en,0,39088|10151291314189991
4,112650472155143_3184289658324527,Facebook,2020-06-09 21:05:36,2021-07-14 01:05:29,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",channeldayton.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/XQrWkYy,https://www.facebook.com/112650472155143/posts...,8258,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,4,1,0,1,39591,"Channel 99.9 Dayton, Ohio",channeldayton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,http://www.channeldayton.com,True,2011-05-27 15:39:28,en,0,39591|3184289658324527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14766,10643211755_10159469653066756,Facebook,2020-06-09 01:25:11,2021-08-31 12:13:38,li

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}"
1,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}"
2,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}"
3,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}"
4,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}"


In [24]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,189346456615_10157352875286616,Facebook,2020-06-09 21:05:38,2021-07-07 12:22:24,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",hot1079.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/452p8Y6,https://www.facebook.com/189346456615/posts/10...,15541,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,1,2,1,1,0,0,428743,HOT 107.9 Syracuse,hot1079syracuse,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,HOT 107.9 | All The Hits For Syracuse\nElvis D...,True,2009-12-04 17:48:04,en,0,428743|10157352875286616,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
1,48406298570_10158377191118571,Facebook,2020-06-09 21:05:38,2021-09-04 02:11:57,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",kiss1023.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/qcie1mk,https://www.facebook.com/48406298570/posts/101...,77246,-0.857143,1.0,1,0,0,0,0,0,0,0,0,0,2,1,1,1,1,0,1,0,0,0,156408,KISS 102-3,Kiss1023,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,The Capital Region's Home for Elvis Duran in t...,True,2009-01-28 22:01:34,en,0,156408|10158377191118571,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
2,838752389553295_2902666793161834,Facebook,2020-06-09 21:05:37,2021-05-13 06:06:25,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",thebeatgulfcoast.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/iCfLMAy,https://www.facebook.com/838752389553295/posts...,1726,-0.928571,1.0,0,0,0,1,0,0,0,0,0,0,1,2,1,2,1,2,2,3,0,1,3261001,The Beat Gulf Coast,TheBeat1073,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Mobile's Home for Hip Hop n' R & B,False,2015-08-31 17:09:00,en,0,3261001|2902666793161834,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
3,11236264990_10151291314189991,Facebook,2020-06-09 21:05:37,2021-07-08 05:06:04,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",wfmf.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/QjnkvRM,https://www.facebook.com/11236264990/posts/101...,9200,-20.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,1,2,1,0,1,39088,102.5 WFMF,1025wfmf,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Baton Rouge's #1 Hit Music station with Kidd K...,True,2008-08-02 02:16:16,en,0,39088|10151291314189991,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
4,112650472155143_3184289658324527,Facebook,2020-06-09 21:05:36,2021-07-14 01:05:29,link,"George Floyd Houston Funeral: Jamie Foxx, Chan...",channeldayton.iheart.com,"After a three-state, five-day journey, George ...","After a three-state, five-day journey, George ...",1.0,http://ihe.art/XQrWkYy,https://www.facebook.com/112650472155143/posts...,8258,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,4,1,0,1,39591,"Channel 99.9 Dayton, Ohio",channeldayton,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,http://www.channeldayton.com,True,2011-05-27 15:39:28,en,0,39591|3184289658324527,"{'Houst

In [25]:
ct.to_csv('a0609.csv')

In [26]:
ct=pd.read_csv('a0609.csv',lineterminator='\n')

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
1,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
2,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
3,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}
4,"{'Houston': 1, 'Texas': 1}","{'Houston': 1, 'Texas': 1}",{}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

0                       [Houston, Texas]
1                       [Houston, Texas]
2                       [Houston, Texas]
3                       [Houston, Texas]
4                       [Houston, Texas]
                      ...               
14762                          [Houston]
14763                   [New York State]
14765                     [Harrison Twp]
14767    [Portland, Ore., SALEM, Oregon]
14769    [Portland, Ore., SALEM, Oregon]
Name: NER-msg, Length: 10174, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

0                         [Houston, Texas, Houston, Texas]
1                         [Houston, Texas, Houston, Texas]
2                         [Houston, Texas, Houston, Texas]
3                         [Houston, Texas, Houston, Texas]
4                         [Houston, Texas, Houston, Texas]
                               ...                        
14762                                   [Houston, Houston]
14763                           [New York State, New York]
14765                                       [Harrison Twp]
14767    [Portland, Ore., SALEM, Oregon, Portland, Ore....
14769    [Portland, Ore., SALEM, Oregon, Portland, Ore....
Name: NER, Length: 10174, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ner_lst = ct_ner['NER'].values

In [37]:
ner_lst

array([list(['houston', 'texas']), list(['houston', 'texas']),
       list(['houston', 'texas']), ..., list(['harrison twp']),
       list(['portland', 'ore.', 'salem', 'oregon']),
       list(['portland', 'ore.', 'salem', 'oregon'])], dtype=object)

In [38]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [39]:
list

['houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'the virgin islands',
 'tx',
 'atlanta',
 'tuscaloosa',
 'america',
 'tuscaloosa',
 'us',
 'us',
 'new york city',
 'us',
 'harrison',
 'franklin township',
 'minneapolis',
 'missouri',
 'tuscaloosa',
 'minnesota',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'nj',
 'nj',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston',
 'texas',
 'the virgin islands',
 'houston',
 'texas',
 'houston',
 'texas',
 'houston

In [40]:
c = collections.Counter()

In [41]:
dict_a=collections.Counter(list).most_common()

In [42]:
dict_a

[('houston', 3801),
 ('minneapolis', 2458),
 ('texas', 1463),
 ('america', 834),
 ('u.s.', 751),
 ('the united states', 492),
 ('minnesota', 405),
 ('us', 379),
 ('new york', 317),
 ('washington', 287),
 ('philadelphia', 258),
 ('pearland', 211),
 ('buffalo', 209),
 ('los angeles', 199),
 ('chicago', 181),
 ('new york city', 179),
 ('d.c.', 168),
 ('portland', 159),
 ('nyc', 129),
 ('virginia', 126),
 ('tx', 125),
 ('brooklyn', 123),
 ('maryland', 117),
 ('france', 106),
 ('california', 103),
 ('alabama', 101),
 ('boston', 93),
 ('georgia', 91),
 ('louisville', 88),
 ('north carolina', 88),
 ('canada', 80),
 ('ohio', 76),
 ('paris', 71),
 ('syria', 70),
 ('nj', 69),
 ('new jersey', 69),
 ('germany', 69),
 ('oregon', 68),
 ('brazil', 67),
 ('mexico', 66),
 ('massachusetts', 65),
 ('atlanta', 64),
 ('athens', 64),
 ('australia', 62),
 ('berlin', 61),
 ('new zealand', 61),
 ('milwaukee', 59),
 ('dc', 58),
 ('florida', 57),
 ('seattle', 57),
 ('ft.', 57),
 ('ireland', 57),
 ('lauderdale', 

In [43]:
dataframe609=pd.DataFrame(dict_a)

In [44]:
len(dict_a)

1959

In [45]:
dataframe609.to_csv('dataframe0609.csv')